<a href="https://colab.research.google.com/github/HanKhanh25/HanKhanh25/blob/main/H%C3%A1n_Duy_Kh%C3%A1nh_20002059.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 27 not upgraded.
Need to get 36.6 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u342-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-j

In [32]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions
# import zipfile
# with zipfile.ZipFile("ml-100k.zip", 'r') as zip_ref:
#     zip_ref.extractall()
  


 








In [33]:
def loadMovieNames():
  movieNames = {}
  with open ("ml-100k/u.item", encoding = "latin1") as f:
    for line in f:
      fields = line.split('|')
      movieNames[int(fields[0])] = fields[1]
      return movieNames
movieNames = loadMovieNames()
print(movieNames) 

{1: 'Toy Story (1995)'}


In [34]:
def parseInput(line):
  fields = line.split() 
  return Row(movieID = int(fields[1]), rating = float(fields[2]))

In [35]:
spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

In [36]:
lines = spark.sparkContext.textFile("ml-100k/u.data")

In [37]:
movies = lines.map(parseInput)
movies.take(10)

[Row(movieID=242, rating=3.0),
 Row(movieID=302, rating=3.0),
 Row(movieID=377, rating=1.0),
 Row(movieID=51, rating=2.0),
 Row(movieID=346, rating=1.0),
 Row(movieID=474, rating=4.0),
 Row(movieID=265, rating=2.0),
 Row(movieID=465, rating=5.0),
 Row(movieID=451, rating=3.0),
 Row(movieID=86, rating=3.0)]

In [38]:
movieDataset = spark.createDataFrame(movies)
movieDataset.take(10)

[Row(movieID=242, rating=3.0),
 Row(movieID=302, rating=3.0),
 Row(movieID=377, rating=1.0),
 Row(movieID=51, rating=2.0),
 Row(movieID=346, rating=1.0),
 Row(movieID=474, rating=4.0),
 Row(movieID=265, rating=2.0),
 Row(movieID=465, rating=5.0),
 Row(movieID=451, rating=3.0),
 Row(movieID=86, rating=3.0)]

In [39]:
averageRatings = movieDataset.groupBy("movieID").avg("rating")
averageRatings.take(10)

[Row(movieID=474, avg(rating)=4.252577319587629),
 Row(movieID=29, avg(rating)=2.6666666666666665),
 Row(movieID=26, avg(rating)=3.452054794520548),
 Row(movieID=964, avg(rating)=3.3333333333333335),
 Row(movieID=65, avg(rating)=3.5391304347826087),
 Row(movieID=191, avg(rating)=4.163043478260869),
 Row(movieID=1224, avg(rating)=2.6666666666666665),
 Row(movieID=558, avg(rating)=3.6714285714285713),
 Row(movieID=1010, avg(rating)=3.25),
 Row(movieID=418, avg(rating)=3.5813953488372094)]

In [40]:
counts = movieDataset.groupBy("movieID").count()
counts.take(10)

[Row(movieID=474, count=194),
 Row(movieID=29, count=114),
 Row(movieID=26, count=73),
 Row(movieID=964, count=9),
 Row(movieID=65, count=115),
 Row(movieID=191, count=276),
 Row(movieID=1224, count=12),
 Row(movieID=558, count=70),
 Row(movieID=1010, count=44),
 Row(movieID=418, count=129)]

In [41]:
averagesAndCounts = counts.join(averageRatings, "movieID")
averagesAndCounts.take(10)

[Row(movieID=474, count=194, avg(rating)=4.252577319587629),
 Row(movieID=29, count=114, avg(rating)=2.6666666666666665),
 Row(movieID=26, count=73, avg(rating)=3.452054794520548),
 Row(movieID=964, count=9, avg(rating)=3.3333333333333335),
 Row(movieID=65, count=115, avg(rating)=3.5391304347826087),
 Row(movieID=191, count=276, avg(rating)=4.163043478260869),
 Row(movieID=1224, count=12, avg(rating)=2.6666666666666665),
 Row(movieID=558, count=70, avg(rating)=3.6714285714285713),
 Row(movieID=1010, count=44, avg(rating)=3.25),
 Row(movieID=418, count=129, avg(rating)=3.5813953488372094)]

In [42]:
sorted_DF = averagesAndCounts.orderBy("avg(rating)")
topTen = sorted_DF.head(10)
for movie in topTen:
  print(movieNames[movie[0]], movie[1], movie[2])
spark.stop() 

KeyError: ignored

# New Section